# Vorverarbeitung

In [11]:
import fitz
import re
import unicodedata

Text aus PDF-Dateien extrahieren

In [12]:
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        doc = fitz.open(pdf_path)
        for page in doc:
            text += page.get_text()
        doc.close()
    except Exception as e:
        print(f"Fehler beim Lesen von {pdf_path}: {e}")
    return text

Bereinigung des Texts von zusätzlichen Leerzeichen, Bindestrichen,...

In [13]:
def clean_text(text):
    text = unicodedata.normalize("NFKC", text)
    text = text.replace('\xad', '')
    text = re.sub(r'(\w+)-\s*\n\s*(\w+)', r'\1\2', text)
    text = text.replace("\n", " ")
    text = ''.join(ch for ch in text if unicodedata.category(ch)[0] != 'C')
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\s+([.,;:!?])', r'\1', text)
    return text.strip()

Aufteilung des Textes in Sätze

In [14]:
def split_sentences(text):
    protected = {
        r'\bz\. *B\.': 'z.B.',
        r'\bu\. *a\.': 'u.a.',
        r'\bu\. *Ä\.': 'u.Ä.',
        r'\bd\. *h\.': 'd.h.',
        r'\bu\. *s\. *w\.': 'usw.',
    }
    for pattern, replacement in protected.items():
        text = re.sub(pattern, replacement, text)
    sentences = re.split(r'(?<=[.!?])\s+(?=[A-ZÄÖÜ])', text)
    return [s.strip() for s in sentences if s.strip()]

Gruppierung der Sätze in Chunks

In [15]:
def group_sentences(sentences, max_length=500):
    chunks, current = [], ""
    for sentence in sentences:
        if len(current) + len(sentence) + 1 <= max_length:
            current += " " + sentence if current else sentence
        else:
            chunks.append(current.strip())
            current = sentence
    if current:
        chunks.append(current.strip())
    return chunks

Kombination der Aufteilung von Sätzen und Chunks

In [16]:
def split_and_chunk_text(text, max_length=500):
    sentences = split_sentences(text)
    return group_sentences(sentences, max_length)

# LLM Modell

Die Klasse definiert zunächst zentrale Parameter wie Host, Port, Timeout und Standardoptionen für die Modellgenerierung (z. B. Temperatur, Top-k, Stopbedingungen). Über die Methode models() kann geprüft werden, welche Modelle auf dem Server verfügbar sind. Mit pull_model() lassen sich Modelle bei Bedarf vom zentralen Ollama-Repository herunterladen und bereitstellen.

Für die Textgenerierung stehen zwei Hauptfunktionen zur Verfügung:

- completion() sendet einfache Prompts an das Modell.

- chat() ermöglicht einen Austausch im Format eines Chatverlaufs mit mehreren Rollen (z. B. Nutzer und Modell).

Beide Methoden nutzen intern api_request(), die die Kommunikation mit dem Ollama-Server übernimmt und dafür sorgt, dass Anfragen entweder als Streaming (Debugzwecken) oder klassisch per JSON beantwortet werden.

Die Antwortverarbeitung erfolgt robust durch zwei weitere Methoden:

- secure_text_response() extrahiert das Ergebnis aus der JSON-Antwort, berechnet die Dauer und die Anzahl generierter Tokens.

- secure_json_response() prüft zusätzlich, ob die Antwort im JSON- oder Markdown-Format vorliegt und verarbeitet ggf. auch eingebettete <think>-Blöcke, falls das Modell seine Begründung mitschickt.

Um sicherzustellen, dass fehlerhafte Kodierungen oder Zeichenfolgen nicht zu Problemen führen, wurde fix_invalid_escapes() ergänzt, die mithilfe der ftfy-Bibliothek Textprobleme korrigiert.

Diese API-Klasse stellt damit eine flexible Schnittstelle dar, um eigene Texte (wie Aussagen aus Nachhaltigkeitsberichten) effizient mit lokalen Sprachmodellen zu analysieren.

In [17]:
import requests
import json
import ftfy
import os

In [18]:
class OllamaApi:

    HOST = "https://f2ki-h100-1.f2.htw-berlin.de"
    PORT = 11435

    TIMEOUT = 120
    STREAM_RESPONSE = False

    THINKING = False

    FALSE_RETURN = {"result": None, "time": 0, "token": 0, "info": {}}

    DEFAULT_OPTIONS = {
        "num_ctx": 2048,        # Default: 2048
        "repeat_last_n": 64,    # Default: 64, 0 = disabled, -1 = num_ctx
        "repeat_penalty": 1.1,  # Default: 1.1
        "temperature": 0.8,     # Default: 0.8
        "seed": 0,              # Default: 0
        "stop": [],             # No default
        "num_predict": -1,      # Default: -1, infinite generation
        "top_k": 40,            # Default: 40
        "top_p": 0.9,           # Default: 0.9
        "min_p": 0.0            # Default: 0.0
    }

    @staticmethod
    def fix_invalid_escapes(s):
        if not isinstance(s, str):
            return s

        try:
            s_re = ftfy.fix_text(s)
            if s_re != s:
                s = s_re
        except Exception as e:
            print(f"Encoding failed for text: {e}")

        return s

    @classmethod
    def models(cls):
        url = f"{cls.HOST}:{cls.PORT}/api/tags"
        headers = {
            "accept": "application/json",
        }
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Request failed with status {response.status_code}: {response.text}")
            return False
        else:
            try:
                json_data = response.json()
                return json_data.get("models", [])

            except json.JSONDecodeError as e:
                print(f"Failed to parse JSON: {e}")
                return False

    @classmethod
    def pull_model(cls, name:str, tag:str):
        url = f"{cls.HOST}:{cls.PORT}/api/pull"
        headers = {
            "Content-Type": "application/json",
            "accept": "application/json"
        }
        payload = {
            "model": f"{name}:{tag}"
        }
        try:
            response = requests.post(url, headers=headers, json=payload, stream=True, timeout=cls.TIMEOUT)
            if response.status_code != 200:
                print(f"Pull request failed with status {response.status_code}: {response.text}")
                return False

            for line in response.iter_lines():
                if line:
                    try:
                        progress = json.loads(line)
                        if 'status' in progress:
                            print(f"Pulling model: {progress['status']}")
                        if progress['status'] == "success":
                            return True
                    except json.JSONDecodeError:
                        continue

            return True
        except Exception as e:
            print(f"Failed to pull model: {e}")
            return False

    @classmethod
    def completion(cls, prompt:str, model="phi4:latest", schema=None, options=None):
        payload = {
            "model": model,
            "prompt" : prompt,
            "options": {
                **cls.DEFAULT_OPTIONS,
                **options
            } if options is not None else cls.DEFAULT_OPTIONS,
        }
        if schema is not None:
            payload["format"] = schema

        return cls.api_request(payload, force_json=False if schema is None else True)

    @classmethod
    def chat(cls, chat, model="phi4:latest", schema=None, options=None):
        payload = {
            "model": model,
            "messages": chat,
            "options": {
                **cls.DEFAULT_OPTIONS,
                **options
            } if options is not None else cls.DEFAULT_OPTIONS,
        }
        if schema is not None:
            payload["format"] = schema

        return cls.api_request(payload, force_json=False if schema is None else True)

    @classmethod
    def api_request(cls, payload, force_json:bool):
        if "messages" in payload:
            # Chat Request
            url = f"{cls.HOST}:{cls.PORT}/api/chat"
        else:
            # Completion Request
            url = f"{cls.HOST}:{cls.PORT}/api/generate"

        headers = {
            "Content-Type": "application/json",
            "accept": "application/json"
        }

        payload = {
            **payload,
            "think": cls.THINKING,
            "stream": cls.STREAM_RESPONSE,
            "keep_alive": "5m"
        }

        try:
            response = requests.post(url, headers=headers, json=payload, stream=cls.STREAM_RESPONSE, timeout=cls.TIMEOUT)

            if cls.STREAM_RESPONSE:
                for line in response.iter_lines(decode_unicode=True):
                    try:
                        chunk = json.loads(line)

                        if 'message' in chunk and 'content' in chunk['message']:
                            content = chunk['message']['content']
                            print(content, end='', flush=True)

                        if 'done' in chunk and chunk['done']:
                            break
                    except json.JSONDecodeError as e:
                        print(f"ERROR: Failed to decode JSON during streaming: {e}")
                return {**cls.FALSE_RETURN, "info": {"error": 'Streaming mode does not retreive a value'}}
            else:
                return cls.secure_json_response(response) if force_json else cls.secure_text_response(response)

        except requests.exceptions.Timeout:
            print(f"ERROR: The request took to long. Adjust the timeout ({cls.TIMEOUT}) as needed")
            return {**cls.FALSE_RETURN, "info": {"error": f"Request timeout ({cls.TIMEOUT}) reached"}}
        except Exception as e:
            print(f"ERROR: Request exception: {e}")
            return {**cls.FALSE_RETURN, "info": {"error": f"Request exception: {e}"}}


    @classmethod
    def secure_json_response(cls, response):

        text_response = cls.secure_text_response(response)

        if text_response.get("result") is None:
            return text_response

        message = str(text_response.get("result"))

        markdown_response = False
        thinking_block = False

        if message.strip().startswith("<think>"):
            print('WARN: Model returned <think> reasoning block before JSON')
            message = re.sub(r"^\s*<think>.*?</think>\s*", "", message, flags=re.DOTALL).strip()
            thinking_block = True

        match = re.search(r'```json(.*?)```', message, re.DOTALL)
        if match:
            # alles außer dem Inhalt von „```json“ bis „```“ entfernen
            print('WARN: Model returned markdown instead of only JSON')
            message = match.group(1).strip()
            markdown_response = True

        try:
            # JSON parsen
            result = json.loads(message)

            # Überschreiben des Textergebnisses mit JSON dict
            text_response["result"] = dict(result)
            text_response["info"] = {
                "thinking": thinking_block,
                "markdown": markdown_response
            }
            return text_response

        except json.JSONDecodeError as e:
            print(f"ERROR: Failed to decode JSON: {e}")
            return {**cls.FALSE_RETURN, "info": {"error": 'JSON decode error on the model\'s response'}}

        except Exception as e:
            print(f"ERROR: Failed to parse JSON: {e}")
            return {**cls.FALSE_RETURN, "info": {"error": str(e)}}

    @classmethod
    def secure_text_response(cls, response):

        try:
            parsed_json = response.json()

            if response.status_code != 200:
                err_msg = parsed_json.get('error', 'Unknown error')
                print(f"ERROR: Request failed with status {response.status_code}: {err_msg}")
                return {**cls.FALSE_RETURN, "info":{"error":err_msg}}

            if 'done' not in parsed_json or parsed_json.get('done') is False:
                print("ERROR: Response has returned but Model didn't complete the answer")
                return {**cls.FALSE_RETURN, "info": {"error": 'Incomplete answer'}}

            # LLM Chat Rückgabe als String
            message = parsed_json.get('message').get('content') if "message" in parsed_json else parsed_json.get('response')

            microseconds_elapsed = parsed_json.get('total_duration')
            seconds_elapsed = round(microseconds_elapsed / 1000000000, 3)
            token_count = parsed_json.get('eval_count')

            return {
                "result": cls.fix_invalid_escapes(message),
                "time": float(seconds_elapsed),
                "token": int(token_count),
                "info": {}
            }

        except json.JSONDecodeError as e:
            print(f"ERROR: Failed to decode JSON: {e}")
            return {**cls.FALSE_RETURN, "info": {"error": 'JSON decode error on the ollama server\'s response'}}

        except Exception as e:
            print(f"ERROR: Failed to parse JSON: {e}")
            return {**cls.FALSE_RETURN, "info": {"error": str(e)}}

# Filterung von Nachhaltigkeitsversprechen mit LLM

Im folgenden Abschnitt wird der vollständige Workflow zur Analyse von PDF-Dokumenten (z. B. Nachhaltigkeits- oder Geschäftsberichte) beschrieben. Ziel ist es, mithilfe eines großen Sprachmodells relevante Nachhaltigkeitsversprechen aus dem Text zu extrahieren. Der Code ist in mehrere Teilfunktionen gegliedert, um Wiederverwendbarkeit und Übersichtlichkeit zu gewährleisten.

Für die spätere Klassifikation sollten die Nachhaltigkeitsberichte folgende Benennung erhalten: Unternehmen_Jahr.pdf

Modell und Prompt wählen

In [19]:
# Modell wählen
model_name = "llama3.1:70b"
print(f"-------\nUsing model: {model_name}")

# Ordnerpfad mit PDFs
pdf_folder_path = "./mercedes"

# Prompt-Vorlage
prompt_template = """
"Extrahiere bitte alle Nachhaltigkeitsversprechen, die ein konkretes Ziel verfolgen, aus folgendem Text. Gib nur den relevanten Text zurück, keine Einleitung oder sonstigen Text:\n",
{text}
"""

-------
Using model: llama3.1:70b


Analyse der PDF-Datei

In [20]:
# PDF-Dateien durchgehen
def is_promise(text):
    negativ_phrase = [
        "keine", "nicht enthalten", "nicht vorhanden", "leider", "keine Angaben",
        "keine relevanten", "keine Informationen", "keine konkreten", "keine expliziten",
        "keine spezifischen Ziele", "nur Begriffserklärungen", "keine Nachhaltigkeitsversprechen",
    ]
    return not any(phrase.lower() in text.lower() for phrase in negativ_phrase)

In [21]:
for filename in os.listdir(pdf_folder_path):
    if filename.endswith(".pdf"):
        full_path = os.path.join(pdf_folder_path, filename)
        print(f"\n---\nVerarbeite Datei: {filename}")

        # Text extrahieren und bereinigen
        raw_text = extract_text_from_pdf(full_path)
        if not raw_text.strip():
            print("PDF enthält keinen extrahierbaren Text.")
            continue

        cleaned_text = clean_text(raw_text)
        chunks = split_and_chunk_text(cleaned_text, max_length=500)
        print(f"PDF in {len(chunks)} Chunks aufgeteilt.")

        all_results = []

        for i, chunk in enumerate(chunks):
            print(f"\n→ Chunk {i+1}/{len(chunks)} wird analysiert...")
            prompt = prompt_template.format(text=chunk)
            try:
                result = OllamaApi.completion(prompt, model=model_name)
                extracted = result.get("result", "").strip()
                if extracted:
                    all_results.append(extracted)
            except Exception as e:
                print(f"Fehler bei der Modellabfrage (Chunk {i+1}): {e}")

        # Speichern extrahierter Inhalte zur Weiterverarbeitung
        promises = [res for res in all_results if is_promise(res)]
        
        with open(f"results_LLM/extracted_{filename}.txt", "w", encoding="utf-8") as f:
            for res in promises:
                f.write(res + "\n")
        # Ergebnisse anzeigen
        print(f"\n### Extrahierte Nachhaltigkeitsversprechen aus {filename}:\n")
        for i, res in enumerate(all_results):
            print(f"--- Chunk {i+1} ---\n{res}\n")



---
Verarbeite Datei: Mercedes-Benz_2023.pdf
PDF in 1979 Chunks aufgeteilt.

→ Chunk 1/1979 wird analysiert...

→ Chunk 2/1979 wird analysiert...

→ Chunk 3/1979 wird analysiert...

→ Chunk 4/1979 wird analysiert...

→ Chunk 5/1979 wird analysiert...

→ Chunk 6/1979 wird analysiert...

→ Chunk 7/1979 wird analysiert...

→ Chunk 8/1979 wird analysiert...

→ Chunk 9/1979 wird analysiert...

→ Chunk 10/1979 wird analysiert...

→ Chunk 11/1979 wird analysiert...

→ Chunk 12/1979 wird analysiert...

→ Chunk 13/1979 wird analysiert...

→ Chunk 14/1979 wird analysiert...

→ Chunk 15/1979 wird analysiert...

→ Chunk 16/1979 wird analysiert...

→ Chunk 17/1979 wird analysiert...

→ Chunk 18/1979 wird analysiert...

→ Chunk 19/1979 wird analysiert...

→ Chunk 20/1979 wird analysiert...

→ Chunk 21/1979 wird analysiert...

→ Chunk 22/1979 wird analysiert...

→ Chunk 23/1979 wird analysiert...

→ Chunk 24/1979 wird analysiert...

→ Chunk 25/1979 wird analysiert...

→ Chunk 26/1979 wird analysiert

KeyboardInterrupt: 

# Überprüfung der Einhaltung der Versprechen

Die Überprüfung der Nachhaltigkeitsversprechen wurde mit ChatGPT durchgeführt, da OllamaApi keinen Internetzugriff besitzt und nicht zur Suche nach Artikeln geeignet ist. Es wurde auch eine Artikelsuche mittels Serpapi versucht, jedoch wurden hiermit keine guten Ziele erreicht.

Damit die Liste an Nachhaltigkeitsverprechen ChatGPT übergeben werden können musste die Liste nochmals kompakter formuliert werden. Hierfür wurde die zuvor generierte Liste an Nachhaltigkeitsversrechen noch einmal der OllamaApi übergeben. Hierdurch wird die Liste gekürzt, indem doppelte Versprechen oder nicht klar definierte Ziele aussortiert werden. Es werden alle Nachhatligkeitsversprechen dem Modell auf einmal übergben. Anschließend kann die enstandene Liste an Versprechen mit ChatGPT überprüft werden.

In [23]:
import os
import csv

# Modell wählen
model_name = "llama3.1:70b"

# Prompt-Vorlage
prompt_template = """
Du bist ein Textanalyse-Tool, das Nachhaltigkeitsversprechen aus Unternehmensdokumenten extrahiert.

Analysiere den folgenden Text und extrahiere **alle Nachhaltigkeitsversprechen** in **klaren, prägnanten Stichpunkten**.

**Wichtige Regeln:**
- Entferne alle doppelten oder sinngemäß gleichen Aussagen (Deduplizierung).
- Formuliere jeden Punkt neutral und sachlich.
- Vermeide Wiederholungen oder Variationen derselben Aussage.
- Achte auf konkrete Absichten, Maßnahmen, Ziele oder Prinzipien im Bereich Umwelt, Soziales oder Unternehmensführung (ESG).
- Gib nur die Versprechen wieder – keine allgemeinen Aussagen oder Beschreibungen.

**Format:**
- Gib die Ergebnisse in einer nummerierten Liste zurück.
- Max. ein Satz pro Stichpunkt.

**Text:**  

{text}
"""

# Eingabe- und Ausgabeordner
input_folder = "results_LLM"
output_folder = "results_promises_chatgpt_format"

# Sicherstellen, dass der Ausgabeordner existiert
os.makedirs(output_folder, exist_ok=True)

# Durchlaufe alle .txt-Dateien im Ordner
for filename in os.listdir(input_folder):
    if filename.endswith(".txt"):
        filepath = os.path.join(input_folder, filename)

        # Dateiinhalt lesen
        with open(filepath, "r", encoding="utf-8") as file:
            text_content = file.read()

        # Prompt erstellen
        prompt = prompt_template.format(text=text_content)
        print(f"\n→ Verarbeite Datei: {filename}")

        try:
            result = OllamaApi.completion(prompt, model=model_name)
            extracted = result.get("result", "").strip()

            if extracted:
                print(f"→ Extraktion erfolgreich.")
                # Jede Zeile des Ergebnisses wird einzeln in CSV geschrieben
                extracted_lines = [line.strip() for line in extracted.split("\n") if line.strip()]
            else:
                print("→ Keine Ergebnisse extrahiert.")
                extracted_lines = ["(keine Versprechen extrahiert)"]

        except Exception as e:
            print(f"Fehler bei Datei {filename}: {e}")
            extracted_lines = ["Fehler bei Modellabfrage"]

        # Ausgabedateiname erzeugen: prepared_<Marke>_<Jahr>.txt
        base_name = os.path.splitext(filename)[0]  # entfernt .txt
        parts = base_name.replace("extracted_", "").replace(".pdf", "").split("_")
        if len(parts) >= 2:
            brand = parts[0]
            year = parts[1]
            txt_name = f"prepared_{brand}_{year}.txt"
        else:
            txt_name = f"prepared_unbekannt.txt"

        txt_path = os.path.join(output_folder, txt_name)

        # Ergebnisse in TXT-Datei schreiben
        with open(txt_path, "w", encoding="utf-8") as txtfile:
            txtfile.write(extracted)

        print(f"→ Ergebnisse gespeichert in: {txt_path}")

print("\n Alle Dateien wurden verarbeitet.")


→ Verarbeite Datei: extracted_Audi_2023.pdf.txt
→ Extraktion erfolgreich.
→ Ergebnisse gespeichert in: results_promises_chatgpt_format/prepared_Audi_2023.txt

→ Verarbeite Datei: extracted_Porsche_2023.pdf.txt
→ Extraktion erfolgreich.
→ Ergebnisse gespeichert in: results_promises_chatgpt_format/prepared_Porsche_2023.txt

 Alle Dateien wurden verarbeitet.


## Prompting Überprüfung Versprechen

Der Prompt zur Überprüfung der Nachhaltigkeitsversprechen sieht wie folgt aus:

**Rolle:**  
Du bist ein kritischer Nachhaltigkeitsanalyst mit Fokus auf ESG-Zielerreichung großer Unternehmen.

**Aufgabe:**  
Bewerte eine Liste von Nachhaltigkeitsversprechen aus dem VW-Nachhaltigkeitsbericht 2023 hinsichtlich ihrer Einhaltung und Realisierbarkeit.

**Anleitung:**  
Für jedes aufgeführte Versprechen führe bitte die folgende Analyse durch:

1. **Statusbewertung**: Wurde das Versprechen bereits
   - vollständig erfüllt,
   - teilweise erfüllt,
   - nicht erfüllt,
   oder
   - liegt die Zielerreichung noch in der Zukunft?

2. **Prognosebewertung (nur falls Zukunftsziel)**: 
   - Wie realistisch ist es, dass VW dieses Ziel in der angestrebten Zeit erreichen wird? (realistisch / unsicher / unrealistisch)

3. **Beleg**: Nutze vertrauenswürdige Quellen (z. B. Zeitungsartikel, ESG-Ratings, Reports von NGOs oder Wirtschaftsanalysten), die **nach 2023 veröffentlicht** wurden. Verwende **keine** Veröffentlichungen der Volkswagen Group oder Volkswagen. Beziehe dich **nur** auf **unabhängige Quellen**. Verlinke jede Quelle direkt.

4. **Ampelstatus**:
   - 🟢 **Grün**: erfüllt oder realistisch
   - 🟡 **Gelb**: teilweise erfüllt oder unsicher
   - 🔴 **Rot**: nicht erfüllt oder unrealistisch

**Format der Antwort:**  
Gib die Ergebnisse bitte in folgender Tabellenstruktur zurück:

| # | Nachhaltigkeitsversprechen | Status | Prognose (falls zutreffend) | Quelle(n) | Ampel |
|---|-----------------------------|--------|------------------------------|-----------|-------|
| 1 | [Versprechen 1]             | ...    | ...                          | ...       | ...   |
| 2 | [Versprechen 2]             | ...    | ...                          | ...       | ...   |

**Zu überprüfende Liste:**  
[extrahierte Versprechen]

Alle Ergebnisse sind im Ordner "results_chatgpt" zu finden

## Beispielergebnis VW:
| # | Nachhaltigkeitsversprechen                                                                                      | Status                                                                                                                                                        | Prognose (falls zutreffend) | Quelle(n)                                                                                                                                                                                                            | Ampel |
| - | --------------------------------------------------------------------------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ----- |
| 1 | **80 % der direkten Lieferanten mit ISO 14001 oder EMAS (Ziel 2023)**                                           | ✅ teilweise erfüllt – VW meldet 80 %, unabhängig nicht verifiziert                                                                                            | –                           | VW-Bericht für Basisjahr 2023 (Quelle intern) – unabhängige Bestätigung fehlt                                                                                                                                        | 🟡    |
| 2 | **85 % der direkten Lieferanten mit ISO 14001 oder EMAS (Ziel 2022)**                                           | ✅ teilweise erfüllt – im Report 2023 wird Anteil mit 85 % angegeben                                                                                           | –                           | wie oben (kein unabhängiger Nachweis gefunden)                                                                                                                                                                       | 🟡    |
| 3 | **Neue Lieferanten mit sozial‑/umweltbewertung: 26 % (2023), 12 % (2022)**                                      | ❌ ungeprüft – keine unabhängige Quelle zur validen Bestätigung dieser Werte                                                                                   | –                           | keine unabhängige Datenerhebung verfügbar                                                                                                                                                                            | 🔴    |
| 4 | **Alle Lieferanten müssen VW‑Nachhaltigkeitsanforderungen erfüllen (A‑ oder B‑Rating vergabefähig)**            | ✅ teilweise – laut Amnesty-Ranking VW nur mit 41/90 Punkten bewertet → viele Defizite in HR-DD                                                                | –                           | Amnesty „Recharge for Rights“ EV‑Ranking 2024 – VW erzielt Score 41/90([Volkswagen Group Annual Report 2024][1], [Industry Intelligence Inc.][2], [Reddit][3])                                                       | 🟡    |
| 5 | **Transparenz in Lieferkette Batterierohstoffe (Kobalt, Lithium, Nickel, Graphit)**                             | ✅ teilweise – VW intern umfangreich aktiv; jedoch laut Amnesty und NGOs weiterhin erhebliche Defizite in Transparenz                                          | –                           | Amnesty‑Report und Analysen zeigen VW sehr schwache Offenlegung/HR‑Due‑Diligence([WIRED][4], [Reddit][3]); Leipzig-Studie kritisiert weiterhin ungenannte Schmelzer, Menschenrechtsrisiken([Universität Leipzig][5]) | 🟡    |
| 6 | **OECD Due Diligence Guidance („Due Diligence Guidance for Responsible Supply Chains of Minerals…“) Umsetzung** | ✅ teilweise – VW adressiert Richtlinien, aber Amnesty-Ranking kritisiert weiterhin oberflächliche Ansätze                                                     | –                           | Amnesty 2024 bewertet VW weiterhin schlecht in HR-DD([Reddit][3], [WIRED][4]); VW-Analysen zeigen Fortschritt, aber kaum Konsequenzen bei Verstößen([Universität Leipzig][5], [Volkswagen Group][6])                 | 🟡    |
| 7 | **Engagement zum Schutz risikogefährdeter Gruppen entlang Lieferkette**                                         | ❌ nicht erfüllt – VW bekam schlechte Bewertung bei Menschenrechts-DD, auch 2024 weiter als "industry worst appraisal" eingestuft                              | –                           | Amnesty-Ranking: VW Score 41/90 zeigt schwaches Engagement für Schutz gefährdeter Gruppen([Reddit][3], [WIRED][4])                                                                                                   | 🔴    |
| 8 | **Verbesserung des sozialen Wohlergehens und Teilnahme an Gesundheits‑ und Sicherheitsmaßnahmen vor Ort**       | ✅ teilweise erfüllt – VW beteiligt sich an GIZ‑Initiative „Cobalt for Development“, aber unabhängige Berichte sehen begrenzten Impact                         | –                           | Leipzig/Afrikastudien analysieren VW‑Projekte kritisch: mögliche Greenwashing‑Risiken, begrenzte direkte Wirkung vor Ort([Universität Leipzig][5])                                                                   | 🟡    |
| 9 | **Dekarbonisierung in der Lieferkette vorantreiben**                                                            | ✅ teilweise – VW fordert CO₂‑Ziele bei Lieferanten und Pilotprojekte (z. B. Recycling), aber keine unabhängige Bewertung über tatsächliche Emissionsreduktion | –                           | VW interne Angaben zu CO₂‑Verträgen und Recyclingpilot, jedoch keine unabhängigen Studien zur Wirkung verfügbar                                                                                                      | 🟡    |

[1]: https://annualreport2024.volkswagen-group.com/sustainability-report/environment/introduction-to-environmental-management.html?utm_source=chatgpt.com "Introduction to environmental management - Volkswagen Group Annual Report 2024"
[2]: https://www.industryintel.com/news/vw-joins-initiative-for-responsible-mining-assurance-will-gradually-apply-irma-standards-to-its-battery-supply-chains-for-raw-materials-such-as-cobalt-lithium-nickel-graphite-irma-established-rigorous-standards-for-responsible-raw-material-extraction-157018415640?utm_source=chatgpt.com "VW joins Initiative for Responsible Mining Assurance, will gradually apply IRMA standards to its battery supply chains for raw materials such as cobalt, lithium, nickel, graphite; IRMA established rigorous standards for responsible raw material extraction"
[3]: https://www.reddit.com/r/EuroEV/comments/1g6dngo?utm_source=chatgpt.com "Amnesty International: New human rights ranking of electric vehicle industry exposes laggards"
[4]: https://www.wired.com/story/the-worlds-biggest-maker-of-evs-has-the-worst-appraisal-of-human-rights?utm_source=chatgpt.com "The World's Biggest Maker of EVs Has the Worst Appraisal of Human Rights"
[5]: https://home.uni-leipzig.de/~afrikastudien/2022/02/17/global-energy-justice-the-case-of-cobalt/?utm_source=chatgpt.com "Global Energy Justice: The Case of Cobalt – Afrikastudien"
[6]: https://www.volkswagen-group.com/en/press-releases/volkswagen-group-publishes-third-responsible-raw-materials-report-17465?utm_source=chatgpt.com "Volkswagen Group publishes third Responsible Raw Materials Report | Volkswagen Group"


Da für Punkt 3 keine unabhängigen Quellen gefunden wurden wurde eine Volkswagen Veröffentlichung als Gundlage für die Einschätzung verwendet:
| #  | Nachhaltigkeitsversprechen                                                           | Status                                                                                                                                                                       | Prognose (falls Zukunftsziel) | Quelle(n)                                                                                                                                                                                                                  | Ampel |
| -- | ------------------------------------------------------------------------------------ | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ----------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | ----- |
| 3  | Anteil neuer Lieferanten mit Sozial‑/ Umweltbewertung: 26 % (2023) bzw. 12 % (2022)  | Ziel war zahlenbasiert (Ist‑Wert): tatsächlich 26  % 2023 (korrekt)                                                                                                          | –                             | Daten aus VW‑Bericht 2023; 2024 keine weitere Zielwert‑Angabe, aber Basis bestätigt ([Volkswagen Group Annual Report 2024][1], [VW Annual Report 2023][3])                                                                 | 🟢    |

[3]: https://annualreport2023.volkswagen-group.com/group-management-report/sustainable-value-enhancement/procurement.html?utm_source=chatgpt.com "Procurement - VW Annual Report 2023"


# Zero-Shot-Klassifikation von Nachhaltigkeitsaussagen mit DeBERTa aus den Nachhaltigkeitsberichten

Der Code nutzt ein mehrsprachiges Zero-Shot-Klassifikationsmodell („MoritzLaurer/mDeBERTa-v3-base-mnli-xnli“), um Textabschnitte (Chunks) aus Nachhaltigkeitsberichten automatisch in vorgegebene Kategorien wie „konkrete Maßnahme“, „vage Behauptung“ oder „fragwürdige Nachhaltigkeitsaussage“ einzuordnen.

Dabei kommt die DeBERTa-Variante mit MNLI/XNLI-Feintuning zum Einsatz, die besonders gut für Klassifikationen in verschiedenen Sprachen geeignet ist. Für jede Aussage wird die wahrscheinlichste Kategorie bestimmt. Ist die Klassifikation unsicher – etwa bei einem niedrigen Score (< 0.5) oder einer geringen Differenz zwischen den Top-Labels (< 0.1) – wird die Aussage zusätzlich mit dem Label „unsicher“ versehen.

Der Ansatz eignet sich insbesondere für große Textmengen aus LLM-generierten Extraktionen und ermöglicht eine erste strukturierte Einordnung von Nachhaltigkeitsversprechen ohne manuelles Labeling.

In [ ]:
from transformers import pipeline
import pandas as pd

c:\Dokumente\Uni Master\SoSe25\Unternehmenssoftware\Projekt\Greenwashing_Detector\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")

# Klassifikationslabels
labels = [
    "konkrete Maßnahme",
    "vage Behauptung",
    "fragwürdige Nachhaltigkeitsaussage"
]

input_folder = "./results_LLM"
output_folder = "./results_classification"
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.startswith("extracted_") and filename.endswith(".txt"):
        filepath = os.path.join(input_folder, filename)

        with open(filepath, "r", encoding="utf-8") as f:
            lines = [line.strip() for line in f if line.strip()]

        if not lines:
            continue

        match = re.match(r"extracted_(.+)_(\d{4})\.pdf\.txt", filename)
        if not match:
            print(f"Dateiname nicht im erwarteten Format: {filename}")
            continue

        company = match.group(1).replace("_", " ")
        year = match.group(2)
        print(f"\n→ Klassifiziere Aussagen für: {company}, {year} ({len(lines)} Aussagen)")

        classification_results = []

        filtered_lines = [
            line for line in lines
            if not line.lower().startswith("ich bin bereit") and "füge den text hinzu" not in line.lower()
        ]

        for chunk in filtered_lines:
            try:
                result = classifier(
                    chunk,
                    candidate_labels=labels,
                    hypothesis_template="Diese Aussage stellt eine {} im Kontext von Nachhaltigkeit dar."
                )

                top_label = result["labels"][0]
                top_score = result["scores"][0]
                second_score = result["scores"][1]
                score_diff = top_score - second_score

                # Unsichere Klassifikation kennzeichnen
                if top_score < 0.5 or score_diff < 0.1:
                    label = "unsicher"
                else:
                    label = top_label

                classification_results.append({
                    "company": company,
                    "year": year,
                    "text": chunk,
                    "label": label,
                    "score": top_score,
                    "score_diff": score_diff,
                    "all_labels": result["labels"],
                    "all_scores": result["scores"]
                })
            except Exception as e:
                print(f"Fehler bei der Klassifikation: {e}")
                continue

        # Als CSV speichern
        df = pd.DataFrame(classification_results)
        df["text_short"] = df["text"].str.slice(0, 150)

        csv_filename = f"{company}_{year}.csv".replace(" ", "_")
        output_path = os.path.join(output_folder, csv_filename)
        df.to_csv(output_path, index=False)

        print(f"Ergebnisse gespeichert in: {output_path}")
        print(df["label"].value_counts())


c:\Dokumente\Uni Master\SoSe25\Unternehmenssoftware\Projekt\Greenwashing_Detector\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\schmu\.cache\huggingface\hub\models--MoritzLaurer--mDeBERTa-v3-base-mnli-xnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu



→ Klassifiziere Aussagen für: Audi, 2023 (305 Aussagen)
Ergebnisse gespeichert in: ./results_classification\Audi_2023.csv
label
konkrete Maßnahme                     241
unsicher                               55
fragwürdige Nachhaltigkeitsaussage      5
vage Behauptung                         4
Name: count, dtype: int64

→ Klassifiziere Aussagen für: Porsche, 2023 (515 Aussagen)
Ergebnisse gespeichert in: ./results_classification\Porsche_2023.csv
label
konkrete Maßnahme                     405
unsicher                               80
vage Behauptung                        18
fragwürdige Nachhaltigkeitsaussage      9
Name: count, dtype: int64

→ Klassifiziere Aussagen für: VolkswagenGroup, 2023 (519 Aussagen)
Ergebnisse gespeichert in: ./results_classification\VolkswagenGroup_2023.csv
label
konkrete Maßnahme                     443
unsicher                               60
fragwürdige Nachhaltigkeitsaussage      9
vage Behauptung                         7
Name: count, dtype: int64


# Evaluation der Klassifikation